In [5]:
import solara as sl
from typing import Dict
import plotly.express as px
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from datetime import datetime
import os


# For hitting the API
base_url = os.environ["DOMINO_KUBECOST_URL"]
assets_url = sl.reactive(f"{base_url}/assets")
allocations_url = sl.reactive(f"{base_url}/allocation")
username = os.environ["DOMINO_KUBECOST_USERNAME"]
pwd = os.environ["DOMINO_KUBECOST_PASSWORD"]
auth = sl.reactive(HTTPBasicAuth(username, pwd))

# For interacting with the different scopes
breakdown_options = ["Execution Type", "Top Projects"]
breakdown_to_param = {
    "Execution Type": "dominodatalab_com_workload_type",
    "Top Projects": "dominodatalab_com_project_name",
    "User": "dominodatalab_com_user_username"
}
breakdown_choice = sl.reactive(breakdown_options[0])

# For granular aggregations
window_options = ["Last 30 days", "Last 15 days", "Last week", "Today"]
window_to_param = {
    "Last 30 days": "30d",
    "Last 15 days": "15d",
    "Last week": "lastweek",
    "Today": "today"
}
window_choice = sl.reactive(window_options[0])


def _format_datetime(dt_str: str) -> str:
    datetime_object = datetime.strptime(dt_str, "%Y-%m-%dT%H:%M:%SZ")
    return datetime_object.strftime("%m/%d %I:%M %p")


def get_cost_per_breakdown() -> Dict[str, float]:
    params = {
        "window": window_to_param[window_choice.value],
        "aggregate": f"label:{breakdown_to_param[breakdown_choice.value]}",
        "accumulate": True
    }

    res = requests.get(allocations_url.value, params=params, auth=auth.value)
    data = res.json()["data"][0]
    return {key: round(data[key]["totalCost"], 2) for key in data if not key.startswith("__")}


def get_overall_cost() -> Dict[str, float]:
    params = {
        "window": window_to_param[window_choice.value],
        "aggregate": "category",
        "accumulate": True
    }

    res = requests.get(assets_url.value, params=params, auth=auth.value)

    data = res.json()["data"][0]
    data.keys()
    
    return {key: round(data[key]["totalCost"], 2) for key in data}


def get_daily_cost() -> pd.DataFrame:
    params = {
        "window": window_to_param[window_choice.value],
        "aggregate":"category"
    }

    res = requests.get(assets_url.value, params=params, auth=auth.value)
    data = res.json()["data"]
    # May not have all historical days
    data = [day for day in data if day]


    day_costs = {
        day["Network"]["start"]: {key: round(day[key]["totalCost"], 2) for key in day} for day in data
    }
    return pd.DataFrame(day_costs).T


def get_execution_cost_table() -> pd.DataFrame:
    params = {
        "window": window_to_param[window_choice.value],
        "aggregate": "label:dominodatalab_com_workload_type,label:dominodatalab_com_starting_user_username,label:dominodatalab_com_project_id",
        # TODO: How do I get all the project IDs (and names)
    #     "filter": 'label[dominodatalab_com_project_id]:"645277142a8f2d12e2e13fca"',
        "accumulate": True,
    }

    res = requests.get(allocations_url.value, params=params, auth=auth.value)
    aloc_data = res.json()["data"][0]

    exec_data = []

    compute_cost_keys = ["cpuCost", "gpuCost", "cpuCostAdjustment", "gpuCostAdjustment"]
    storage_cost_keys = ["pvCost", "ramCost", "pvCostAdjustment", "ramCostAdjustment"]

    keys = list(aloc_data.keys())
    keys = [key for key in keys if not key.startswith("__")]
    for key in keys:
        workload_type, username, project_id = key.split("/")
        key_data = aloc_data[key]
        compute_cost = round(sum([key_data[k] for k in compute_cost_keys]), 2)
        storage_cost = round(sum([key_data[k] for k in storage_cost_keys]), 2)
        exec_data.append({
            "TYPE": workload_type,
            "USER": username,
            "START": key_data["start"],
            "END": key_data["end"],
            "COMPUTE": compute_cost,
            "STORAGE": storage_cost,
            "ID": project_id,

        })
    df = pd.DataFrame(exec_data)
    df["START"] = df["START"].apply(_format_datetime)
    df["END"] = df["END"].apply(_format_datetime)
    return df


@sl.component()
def Executions() -> None:
    sl.Markdown("## Executions")
    df = get_execution_cost_table()
    sl.DataFrame(df)


@sl.component()
def DailyCostBreakdown() -> None:
    df = get_daily_cost()
    fig = px.bar(
        df,
        labels={
             "index": "Date",
             "value": "Cost ($)",
         },
        title="Overall Cost (Cumulative)",
        color_discrete_sequence=px.colors.qualitative.Set3
    )
    sl.FigurePlotly(fig)
    
    
@sl.component()
def SingleCost(name: str, cost: float) -> None:
    with sl.Column():
        sl.Text(f"${str(cost)}")
        sl.Text(name)
    
    
@sl.component()
def TopLevelCosts() -> None:
    costs = get_overall_cost()
    with sl.Card():
        with sl.Columns([2, 1, 1, 1]):
            SingleCost("Total", round(sum(list(costs.values())), 2))
            for name, cost in costs.items():
                SingleCost(name, cost)
        

@sl.component()
def OverallCosts() -> None:
    with sl.Column():
        TopLevelCosts()
        DailyCostBreakdown()
        Executions()
    
    
@sl.component()
def CostBreakdown() -> None:
    with sl.Card("Cost Usage - Breakdown"):
        sl.Select(label="breakdown", value=breakdown_choice, values=breakdown_options) 
        costs = get_cost_per_breakdown()
        option = {
#           "title": {
#             "text": 'Executions'
#           },
          "tooltip": {
            "trigger": 'axis',
            "axisPointer": {
              "type": 'shadow'
            }
          },
          "legend": {},
          "grid": {
            "left": '3%',
            "right": '4%',
            "bottom": '3%',
            "containLabel": True
          },
          "xAxis": {
            "type": 'value',
            "boundaryGap": [0, 0.01]
          },
          "yAxis": {
            "type": 'category',
            "data": list(costs.keys())
          },
          "series": [
            {
              "type": 'bar',
              "data": list(costs.values())
            },
          ]
        }
        print(costs)
        # TODO: Need to keep this commented out because it breaks the jupyter session? Something wrong with solara
#         sl.FigureEcharts(option)
    

@sl.component()
def Page() -> None:
    sl.Title("Cost Analysis")
    with sl.Column(style="width:15%"):
        sl.Select(label="Window", value=window_choice, values=window_options) 
    with sl.Columns([2, 3]):
        CostBreakdown()
        OverallCosts()
    
    
Page()

{'Batch': 0.01, 'ModelAPI': 43.0, 'Workspace': 0.01}
{'Batch': 0.01, 'ModelAPI': 43.0, 'Workspace': 0.01}


Cannot show ipywidgets in text

In [12]:
sl.Select?

In [14]:
from datetime import datetime

aloc_url = allocations_url.value

aloc_params = {
    "window": "30d",
    "aggregate": "label:dominodatalab_com_organization_name,label:dominodatalab_com_workload_type,label:dominodatalab_com_starting_user_username,label:dominodatalab_com_project_id",
    # TODO: How do I get all the project IDs (and names)
#     "filter": 'label[dominodatalab_com_project_id]:"645277142a8f2d12e2e13fca"',
    "accumulate": True,
}

aloc_res = requests.get(aloc_url, params=aloc_params, auth=auth.value)
aloc_data = aloc_res.json()["data"][0]



In [15]:
aloc_data.keys()

dict_keys(['__idle__', '__unallocated__/Batch/system-test/64527e5d5be938057548b3c5', '__unallocated__/ModelAPI/system-test/6452770c2a8f2d12e2e13fbb', '__unallocated__/ModelAPI/system-test/645277142a8f2d12e2e13fca', '__unallocated__/ModelAPI/system-test/6452771d2a8f2d12e2e13fdd', '__unallocated__/ModelAPI/system-test/6452772c2a8f2d12e2e13ff2', '__unallocated__/ModelAPI/system-test/64527df95be938057548b365', '__unallocated__/ModelAPI/system-test/64527e015be938057548b374', '__unallocated__/ModelAPI/system-test/64527e0c5be938057548b387', '__unallocated__/ModelAPI/system-test/64527e1e5be938057548b39c', '__unallocated__/Workspace/integration-test/644bf0848a2efe597b4ccfe1', '__unallocated__/__unallocated__/__unallocated__/__unallocated__'])

In [12]:
exec_data = []

compute_cost_keys = ["cpuCost", "gpuCost", "cpuCostAdjustment", "gpuCostAdjustment"]
storage_cost_keys = ["pvCost", "ramCost", "pvCostAdjustment", "ramCostAdjustment"]


def format_datetime(dt: str) -> str:
    datetime_object = datetime.strptime(dt, "%Y-%m-%dT%H:%M:%SZ")
    return datetime_object.strftime("%m/%d %I:%M %p")
    

keys = list(aloc_data.keys())
keys = [key for key in keys if not key.startswith("__")]
for key in keys:
    workload_type, username, project_id = key.split("/")
    key_data = aloc_data[key]
    compute_cost = round(sum([key_data[k] for k in compute_cost_keys]), 2)
    storage_cost = round(sum([key_data[k] for k in storage_cost_keys]), 2)
    exec_data.append({
        "TYPE": workload_type,
        "USER": username,
        "START": key_data["start"],
        "END": key_data["end"],
        "COMPUTE": compute_cost,
        "STORAGE": storage_cost,
        "ID": project_id,
        
    })
df = pd.DataFrame(exec_data)
df["START"] = df["START"].apply(format_datetime)
df["END"] = df["END"].apply(format_datetime)

In [4]:
aloc_data['Batch/system-test/64527e5d5be938057548b3c5']

{'name': 'Batch/system-test/64527e5d5be938057548b3c5',
 'properties': {'cluster': 'cluster-one',
  'node': 'ip-10-0-87-163.us-west-2.compute.internal',
  'controller': 'run-64527e655be938057548b3cd',
  'controllerKind': 'job',
  'namespace': 'domino-compute',
  'pod': 'run-64527e655be938057548b3cd-nlmdp',
  'providerID': 'i-00817036ba564af4a'},
 'window': {'start': '2023-04-28T00:00:00Z', 'end': '2023-05-09T00:00:00Z'},
 'start': '2023-05-03T15:30:00Z',
 'end': '2023-05-03T15:35:00Z',
 'minutes': 5.0,
 'cpuCores': 1.61,
 'cpuCoreRequestAverage': 1.61,
 'cpuCoreUsageAverage': 0.0,
 'cpuCoreHours': 0.134167,
 'cpuCost': 0.004244,
 'cpuCostAdjustment': 0.0,
 'cpuEfficiency': 0.0,
 'gpuCount': 0.0,
 'gpuHours': 0.0,
 'gpuCost': 0.0,
 'gpuCostAdjustment': 0.0,
 'networkTransferBytes': 0.0,
 'networkReceiveBytes': 0.0,
 'networkCost': 0.0,
 'networkCrossZoneCost': 0.0,
 'networkCrossRegionCost': 0.0,
 'networkInternetCost': 0.0,
 'networkCostAdjustment': 0.0,
 'loadBalancerCost': 0.0,
 'load